# ACRO Demonstration

In [12]:
import os
import sys
import pandas as pd
import numpy as np

In [13]:
# uncomment this line if acro is not installed
# ie you are in development mode
# sys.path.insert(0, os.path.abspath(".."))

In [14]:
from acro import ACRO, add_constant, utils

### Instantiate ACRO

In [15]:
acro = ACRO()

INFO:acro:config: {'safe_threshold': 10, 'safe_dof_threshold': 10, 'safe_nk_n': 2, 'safe_nk_k': 0.9, 'safe_pratio_p': 0.1, 'check_missing_values': False}


### Load test data
The dataset used in this notebook is the nursery dataset from OpenML.  
- The dataset can be read directly from OpenML using the code commented in the next cell.
- In this version, it can be read directly from the local machine if it has been downloaded. 
- The code below reads the data from a folder called "data" which we assume is at the same level as the folder where you are working.
- The path might need to be changed if the data has been downloaded and stored elsewhere.
 - for example use:  
    path = os.path.join("data", "nursery.arff")  
    if the data is in a sub-folder of your work folder

In [16]:
from scipy.io.arff import loadarff

path = os.path.join("../data", "nursery.arff")
data = loadarff(path)
df = pd.DataFrame(data[0])
df = df.select_dtypes([object])
df = df.stack().str.decode("utf-8").unstack()
df.rename(columns={"class": "recommend"}, inplace=True)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/nursery.arff'

### Convert 'more than 3' children to random between 4 and 10
Change the children column from categorical to numeric in order to be able to test some of the ACRO functions that require a numeric feature

In [11]:
print(f' children column entries in raw file {df.children.unique()}')

NameError: name 'df' is not defined

In [ ]:
    
df["children"].replace(to_replace={"more": "4"}, inplace=True)
df["children"] = pd.to_numeric(df["children"])

df["children"] = df.apply(
    lambda row: row["children"]
    if row["children"] in (1, 2, 3)
    else np.random.randint(4, 10),
    axis=1,
)

# Examples of producing tabular output
We rely on the industry-standard package **pandas** for tabulating data.  
In the next few examples we show:
- first, how a researcher would normally make a call in pandas, saving the results in a variable that they can view on screen (or save to file?)
- then how the call is identical in SACRO, except that:
  - "pd" is replaced by "acro"
  - the researcher immediately sees a copy of what the TRE output checker will see.
  

### Pandas crosstab
This is an example of crosstab using pandas.  
We first make the call, then the second line print the outputs to screen.

In [ ]:
table = pd.crosstab(df.recommend, df.parents)
print(table)

### ACRO crosstab
This is an example of crosstab using ACRO.  
The INFO lines show the researcher what will be reported to the output checkers.
Then the (suppressed as necessary)  table is shown via. the print command as before.

In [ ]:
safe_table = acro.crosstab(df.recommend, df.parents)
print(safe_table)

### ACRO crosstab with aggregation function

In [ ]:
safe_table = acro.crosstab(df.recommend, df.parents, values=df.children, aggfunc="mean")
print(safe_table)

### ACRO pivot_table
This is an example of pivot table using ACRO.  
- Some researchers may prefer this to using crosstab.   
- Again the call syntax is identical to the pandas "pd.pivot_table"
- in this case the output is non-disclosive

In [ ]:
table = acro.pivot_table(
    df, index=["parents"], values=["children"], aggfunc=["mean", "std"]
)
print(table)

# Regression examples using ACRO

Again there is an industry-standard package in python, this time called **statsmodels**.
- The examples below illustrate the use of the ACRO wrapper standard statsmodel functions
- Note that statsmodels can be called using an 'R-like' format (using an 'r' suffix on the command names)
- most statsmodels functiobns return a "results object" which has a "summary" function that produces printable/saveable outputs 

### Start by manipulating the nursery data to get two numeric variables
- The 'recommend' column is converted to an integer scale

In [ ]:
df["recommend"].replace(
    to_replace={
        "not_recom": "0",
        "recommend": "1",
        "very_recom": "2",
        "priority": "3",
        "spec_prior": "4",
    },
    inplace=True,
)
df["recommend"] = pd.to_numeric(df["recommend"])

new_df = df[["recommend", "children"]]
new_df = new_df.dropna()

### ACRO OLS 
This is an example of ordinary least square regression using ACRO.  
- Above recommend column was converted form categorical to numeric. 
- Now we perform a the linear regression between recommend and children. 
- This version includes a constant (intercept)
- This is just to show how the regression is done using ACRO.  
- **No correlation is expected to be seen by using these variables**

In [ ]:
y = new_df["recommend"]
x = new_df["children"]
x = add_constant(x)

results = acro.ols(y, x)
results.summary()

### ACRO OLSR
This is an example of ordinary least squares regression using the 'R-like' statsmodels api, i.e. from a formula and dataframe using ACRO 

In [ ]:
results = acro.olsr(formula="recommend ~ children", data=new_df)
print(results.summary())

### ACRO Probit
This is an example of probit regression using ACRO  
We use a different combination of variables from the original dataset.

Again, we support the use of R-lijke formulas - because we suprort R 

In [ ]:
new_df = df[["finance", "children"]]
new_df = new_df.dropna()

y = new_df["finance"].astype("category").cat.codes  # numeric
y.name = "finance"
x = new_df["children"]
x = add_constant(x)

results = acro.probit(y, x)
print(results.summary())

### ACRO Logit
This is an example of logistic regression using ACRO using the statmodels function

In [ ]:
results = acro.logit(y, x)
results.summary()

# ACRO functionality to let users manage their outputs

### 1: List current ACRO outputs
This is an example of using the print_output function to list all the outputs created so far

In [ ]:
acro.print_outputs()

### 2: Remove some ACRO outputs before finalising 
This is an example of deleting some of the ACRO outputs.  
The name of the output that needs to be removed should be passed to the function remove_output.  
- Currently, all outputs names contain timestamp; that is the time when the output was created.  
- The output name can be taken from the outputs listed by the print_outputs function, 
- or by listing the results and choosing the specific output that needs to be removed

In [ ]:

acro.remove_output("output_0_2023-06-19-13454368")


### 3: Rename ACRO outputs before finalising
This is an example of renaming the outputs to provide a more descriptive name.  
The timestamp associated with the output name will not get overwritten

In [ ]:
acro.rename_output("output_2_2023-06-19-13461618", "pivot_table")

### 4: Add a comment to output
This is an example to add a comment to outputs.  
It can be used to provide a description or to pass additional information to the output checkers.

In [ ]:
firstitem= list(acro.results.keys())[0]
acro.add_comments(firstitem, "Please let me have this data.")
acro.add_comments(firstitem, "6 cells were supressed in this table")

### 5: Add an unsupported output to the list of outputs
This is an example to add an unsupported outputs (such as images) to the list of outputs

In [ ]:
acro.custom_output(
    "XandY.jfif", "This output is an image showing the relationship between X and Y"
)

## 6 (the big one) Finalise ACRO
This is an example of the function _finalise()_ which the users must call at the end of each session.  
- It takes each output and saves it to a CSV file.    
- It also saves the SDC analysis for each output to a json file or Excel file  
  (depending on the extension of the name of the file provided as an input to the function)

In [ ]:
output = acro.finalise("test_results.json")

# Over to Ben